In [ ]:
import kagglehub
import shutil
import os

# دانلود فایل
default_path = kagglehub.dataset_download("valentynsichkar/traffic-signs-dataset-in-yolo-format")

# مسیر دلخواه برای ذخیره
desired_path = "/content/my_custom_folder"

# اطمینان از وجود مسیر دلخواه
os.makedirs(desired_path, exist_ok=True)

# انتقال فایل‌ها به مسیر دلخواه
for file_name in os.listdir(default_path):
    shutil.move(os.path.join(default_path, file_name), desired_path)

print("Files moved to:", desired_path)


In [ ]:
import os
import numpy as np
from shutil import copyfile

# Step 1: Format Data Folder
def formatData(train_ratio, test_ratio, new_path="/content/my_custom_folder/datasets", root_path="/content/my_custom_folder/ts/ts"):
    image_folder = "images"
    label_folder = "labels"

    for type_data in ["train", "valid", "test"]:
        os.makedirs(os.path.join(new_path, image_folder, type_data), exist_ok=True)
        os.makedirs(os.path.join(new_path, label_folder, type_data), exist_ok=True)

    image_files = [f for f in os.listdir(root_path) if f.endswith('.jpg')]

    # Shuffle the files
    np.random.shuffle(image_files)

    # Define Size Of Each Folder
    train_size = int(train_ratio * len(image_files))
    valid_size = int((1 - (train_ratio + test_ratio)) * len(image_files))

    train_files = image_files[:train_size]
    valid_files = image_files[train_size:train_size+valid_size]
    test_files = image_files[train_size+valid_size:]

    # Move image and label files to new path
    for file in train_files:
        copyfile(os.path.join(root_path, file), os.path.join(new_path, image_folder, "train", file))
        copyfile(os.path.join(root_path, file.replace('.jpg', '.txt')), os.path.join(new_path, label_folder, "train", file.replace('.jpg', '.txt')))

    for file in valid_files:
        copyfile(os.path.join(root_path, file), os.path.join(new_path, image_folder, "valid", file))
        copyfile(os.path.join(root_path, file.replace('.jpg', '.txt')), os.path.join(new_path, label_folder, "valid", file.replace('.jpg', '.txt')))

    for file in test_files:
        copyfile(os.path.join(root_path, file), os.path.join(new_path, image_folder, "test", file))
        copyfile(os.path.join(root_path, file.replace('.jpg', '.txt')), os.path.join(new_path, label_folder, "test", file.replace('.jpg', '.txt')))

formatData(0.6, 0.2)  # Train: 60%, Valid: 20%, Test: 20%

# Step 2: Install and Load YOLOv8
!pip install ultralytics
from ultralytics import YOLO
from ultralytics import settings
settings.update({"wandb": False})

# Load a YOLOv8 model
model = YOLO("yolov8n.pt")  # YOLOv8 nano model for faster training

# Step 3: Config for Training
config_content = """path: /content/my_custom_folder/datasets
train: images/train
val: images/valid
test:  # test images (optional)

# Classes
names:
  0: prohibitory
  1: danger
  2: mandatory
  3: other
"""
with open("/content/my_custom_folder/train_config.yaml", "w") as fw:
    fw.write(config_content)

# Train the model
model.train(data="/content/my_custom_folder/train_config.yaml", epochs=100, batch=-1)

# Step 4: Validate Model
metrics = model.val(split='val')

# Step 5: Predict on Testset
results = model.predict("/content/my_custom_folder/datasets/images/test/", save=True, conf=0.5)

# Step 6: Visualize Results
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

save_dir = results[0].save_dir
fig, axes = plt.subplots(2, 4, figsize=(30, 10))
img_paths = [os.path.join(save_dir, x) for x in os.listdir(save_dir) if x.endswith('.jpg')]
np.random.shuffle(img_paths)

for i in range(8):
    img = mpimg.imread(img_paths[i])
    axes[int(i/4), i%4].imshow(img)
    axes[int(i/4), i%4].axis('off')

# Step 7: Evaluate on Testset
config_content = """path: /content/my_custom_folder/datasets
train:
val:
test: images/test

# Classes
names:
  0: prohibitory
  1: danger
  2: mandatory
  3: other
"""
with open("/content/my_custom_folder/test_config.yaml", "w") as fw:
    fw.write(config_content)

metrics = model.val(split='test', data='/content/my_custom_folder/test_config.yaml')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 24.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 109MB/s]


Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/my_custom_folder/train_config.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 20.5MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        148        243      0.952      0.898      0.959      0.801
           prohibitory         69         99      0.955       0.98      0.994      0.819
                danger         43         53      0.962      0.962      0.985      0.861
             mandatory         30         32      0.934      0.881      0.957      0.822
                 other         49         59      0.958      0.768      0.902      0.702
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train2

image 1/149 /content/my_custom_folder/datasets/images/test/00010.jpg: 384x640 1 other, 36.2ms
image 2/149 /content/my_custom_folder/datasets/images/test/00012.jpg: 384x640 1 prohibitory, 6.6ms
image 3/149 /content/my_custom_folder/datasets/images/test/00014.jpg: 384x640 (no detections), 6.6ms
image 4/149 /content/my_custom_folder/datasets/images/test/00015.jpg: 384x640 1 danger, 7.8ms
image 5/149 /content/my_custom_folder/datasets/i

val: Scanning /content/my_custom_folder/datasets/labels/test... 149 images, 0 backgrounds, 0 corrupt: 100%|██████████| 149/149 [00:00<00:00, 1841.42it/s]

val: New cache created: /content/my_custom_folder/datasets/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.07s/it]


                   all        149        236      0.914      0.869      0.932      0.758
           prohibitory         81        120      0.941      0.992      0.993      0.852
                danger         35         40      0.975       0.95      0.954      0.816
             mandatory         26         29      0.873      0.714      0.867       0.65
                 other         37         47      0.865       0.82      0.915      0.713
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train4
